# CSE-CIC-IDS 2017 K Nearest Neighbors Classifier

In [1]:
model_id = "knearestneighbors-uniform"

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import glob

In [3]:
NOTEBOOK_PATH = "C:/Users/Xetrov/Desktop/SciFair20/Code/"

In [4]:
x_scaled = pd.read_csv(NOTEBOOK_PATH + "IDS2017/x_scaled_powertransform.csv")

y_df_enc = pd.read_csv(NOTEBOOK_PATH + "IDS2017/y_grouped_1henc.csv")

# Split data into train and test

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
x_train, x_valtest, y_train, y_valtest = train_test_split(x_scaled, y_df_enc, test_size = 0.4, random_state = 42)

In [9]:
x_val, x_test, y_val, y_test = train_test_split(x_valtest, y_valtest, test_size = 0.5, random_state = 42)

In [10]:
del x_valtest 
del y_valtest

In [11]:
print("Train:")
print(y_train.sum(axis=0))
print("\nValidation:")
print(y_val.sum(axis=0))
print("\nTest:")
print(y_test.sum(axis=0))

Train:
BENIGN          1364100
Botnet             1185
Brute Force        8273
DoS/DDoS         228292
Infiltration         24
PortScan          95267
Web Attack         1304
dtype: int64

Validation:
BENIGN          454207
Botnet             397
Brute Force       2810
DoS/DDoS         76510
Infiltration         7
PortScan         31787
Web Attack         431
dtype: int64

Test:
BENIGN          454790
Botnet             384
Brute Force       2752
DoS/DDoS         75897
Infiltration         5
PortScan         31876
Web Attack         445
dtype: int64


# Train model

In [5]:
from sklearn.neighbors import KNeighborsClassifier

In [12]:
clf = KNeighborsClassifier(n_neighbors=5, weights='uniform', n_jobs=-1).fit(x_train, y_train)

**Save model**

In [13]:
import pickle

In [14]:
import time

model_filename = model_id  + " [" + time.strftime("%Y%m%d %H%M") + "]"

In [15]:
save_file = open(NOTEBOOK_PATH + "Models/" + model_filename + ".pkl", "wb")
saved_model = pickle.dump(clf, save_file)
save_file.close()

**Test model**

In [16]:
pred = clf.predict(x_val)

In [17]:
pred_index = list(pred.argmax(1))
y_index = list(np.argmax(y_val.to_numpy(), axis=1))

atktypes = sorted(y_val.columns)
pred_series = pd.Series(pred_index, name="Pred").replace(dict(enumerate(atktypes)))
y_series = pd.Series(y_index, name="Actual").replace(dict(enumerate(atktypes)))

matrix = pd.crosstab(pred_series, y_series, margins=True)
matrix

Actual,BENIGN,Botnet,Brute Force,DoS/DDoS,Infiltration,PortScan,Web Attack,All
Pred,,,,,,,,
BENIGN,453759,100,4,54,5,3,6,453931
Botnet,119,297,0,0,0,0,0,416
Brute Force,5,0,2806,0,0,0,0,2811
DoS/DDoS,127,0,0,76454,0,5,4,76590
Infiltration,0,0,0,0,2,0,0,2
PortScan,193,0,0,1,0,31775,0,31969
Web Attack,4,0,0,1,0,4,421,430
All,454207,397,2810,76510,7,31787,431,566149


# Test Set

In [18]:
from sklearn.metrics import f1_score

In [19]:
pred = clf.predict(x_test)
pred_index = list(pred.argmax(1))
y_index = list(np.argmax(y_test.to_numpy(), axis=1))
f1_micro = f1_score(y_index, pred_index, average='micro')
f1_macro = f1_score(y_index, pred_index, average='macro')

print(f1_micro)
print(f1_macro)

0.9988995829719738
0.867243244598331


In [20]:
atktypes = sorted(y_val.columns)
pred_series = pd.Series(pred_index, name="Pred").replace(dict(enumerate(atktypes)))
y_series = pd.Series(y_index, name="Actual").replace(dict(enumerate(atktypes)))

matrix = pd.crosstab(pred_series, y_series, margins=True)
matrix.to_csv(NOTEBOOK_PATH + "Confusion Matrices/test_" + model_filename + ".csv")
matrix

Actual,BENIGN,Botnet,Brute Force,DoS/DDoS,Infiltration,PortScan,Web Attack,All
Pred,,,,,,,,
BENIGN,454351,87,9,60,4,5,4,454520
Botnet,100,297,0,0,0,0,0,397
Brute Force,8,0,2743,0,0,0,1,2752
DoS/DDoS,134,0,0,75832,0,6,2,75974
Infiltration,0,0,0,0,1,0,0,1
PortScan,194,0,0,3,0,31864,0,32061
Web Attack,3,0,0,2,0,1,438,444
All,454790,384,2752,75897,5,31876,445,566149


# Log results

In [21]:
model_log = open(NOTEBOOK_PATH + "model_log.txt", "a")

model_log.write("\n" + model_filename)
model_log.write("\n\tF1 Micro: " + str(f1_micro))
model_log.write("\n\tF1 Macro: " + str(f1_macro))

model_log.close()